## Initialization

In [ ]:
!git clone https://git.ecker.tech/mrq/ai-voice-cloning/
%cd ai-voice-cloning
!apt install python3.8-venv
!python -m venv venv
!source ./venv/bin/activate
!git clone https://git.ecker.tech/mrq/DL-Art-School dlas
!python -m pip install --upgrade pip
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116
!python -m pip install -r ./dlas/requirements.txt
!python -m pip install -r ./requirements.txt

# Update Repos

In [ ]:
%cd /content/ai-voice-cloning/dlas
!git reset --hard HEAD
!git pull
%cd ..
!git reset --hard HEAD
!git pull


# Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/ai-voice-cloning
!rm -r ./training
!ln -s /content/drive/MyDrive/training/

## Running

In [ ]:
%cd /content/ai-voice-cloning
!source ./venv/bin/activate

import os
import sys

sys.argv = [""]
sys.path.append('./src/')

if 'TORTOISE_MODELS_DIR' not in os.environ:
	os.environ['TORTOISE_MODELS_DIR'] = os.path.realpath(os.path.join(os.getcwd(), './models/tortoise/'))

if 'TRANSFORMERS_CACHE' not in os.environ:
	os.environ['TRANSFORMERS_CACHE'] = os.path.realpath(os.path.join(os.getcwd(), './models/transformers/'))

from utils import *
from webui import *

args = setup_args()

webui = setup_gradio()
tts = setup_tortoise()
webui.launch(share=True, prevent_thread_lock=True, height=1000)
webui.block_thread()

# Fallback Training

In [ ]:
# This is in case you can't get training through the web UI
%cd /content/ai-voice-cloning
!python ./dlas/codes/train.py -opt ./training/finetune.yaml

## Exporting

In [ ]:
%cd /content/ai-voice-cloning
!apt install -y p7zip-full
from datetime import datetime
timestamp = datetime.now().strftime('%m-%d-%Y_%H:%M:%S')
!mkdir -p "../{timestamp}/results"
!mv ./results/* "../{timestamp}/results/."
!mv ./training/* "../{timestamp}/training/."
!7z a -t7z -m0=lzma2 -mx=9 -mfb=64 -md=32m -ms=on "../{timestamp}.7z" "../{timestamp}/"
!ls ~/
!echo "Finished zipping, archive is available at {timestamp}.7z"